Разделение новостей на категории при помощи векторных эмбеддингов и алгоритма кластеризации

In [2]:
import pandas as pd

df = pd.read_csv('all_events.csv')
df.head()

,Источник,Дата,Значимость,Категория,Заголовок,Новость
0,Эксперт РА,21.10.2014,4,инвестиции,«Эксперт 400»: для роста капитализации российс...,"Согласно исследованию, в составе 200 крупнейши..."
1,Эксперт РА,19.02.21,3,рейтинг,«Эксперт РА» присвоил рейтинг компании «Газпро...,ПАО «Газпром» – глобальная энергетическая комп...
2,Эксперт РА,18.06.21,3,рейтинг,«Эксперт РА» присвоил рейтинг компании «ГАЗПРО...,Группа Газпром (далее – Группа) - одна из круп...
3,Коммерсант,27.12.21,4,добыча газа,Европа «перебрала» с запасами,Последнюю неделю европейский спотовый рынок бо...
4,Коммерсант,28.12.21,5,поставки газа,«Газпром» восьмой день не бронирует мощности г...,«Газпром» на дополнительной ночной сессии 28 д...


Для начала преобразуем текст новостей в векторные эмбеддинги при помощи TF-IDF векторов. 

In [4]:
import string

import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

def process_text(text):
    return [word for word in word_tokenize(text.lower()) if word not in string.punctuation]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
df['Новость'] = df['Новость'].apply(process_text)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
import numpy as np
from scipy.sparse import hstack

In [10]:
vectors = TfidfVectorizer()
X = (df['Новость']).astype(str)
X = vectors.fit_transform(X)

In [11]:
X

<89x7113 sparse matrix of type '<class 'numpy.float64'>'
	with 19281 stored elements in Compressed Sparse Row format>

In [22]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5).fit_predict(X)

Сравним результаты обучения без учителя с истинной разметкой (с вручную подобранными категориями новостей)

In [23]:
df['y'] = kmeans
for i in range(5):
    print('Номер кластера: ', i)
    print(df.loc[df['y'] == i]['Категория'].to_list())
    print()

Номер кластера:  0
['рейтинг', 'рейтинг']

Номер кластера:  1
['события', 'поставки газа', 'санкции', 'санкции', 'санкции', 'санкции', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'оплата газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'санкции', 'поставки газа']

Номер кластера:  2
['инвестиции', 'рейтинг', 'добыча газа', 'поставки газа', 'поставки газа', 'поставки газа', 'инвестиции', 'поставки газа', 'биржа', 'добыча газа', 'поставки газа', 'поставки газа', 'поставки газа', 'поставки газа', 'санкции', 'поставки газа', 'поставки газа', 'поставки газа', 'добыча газа', 'биржа', 'поставк

Можем заметить, что очень хорошо получилось определить большинство кластеров: 
- 0: рейтинг
- 1: поставки газа
- 3: различные события, совет директоров на самом деле тоже можно было отнести к той же категории
- 4: санкции

В целом даже мне было сложно отделять новости о санкциях от новостей о проблемах с поставками газа, поэтому путаница тут очень объяснимая. 

В кластер под номером 2 были отнесены все новости из категории "биржа", но при этом намешалось еще много других новостей

Интересно также посмотреть на новости, которые были неправильно распределены. 

In [27]:
print('Заголовки новостей, которые по ошибке попали в раздел "поставки газа": ')
print(df.loc[(df['y'] == 1) & (df['Категория'] != 'поставки газа')]['Заголовок'].to_list())
print()
print('Заголовки новостей, которые по ошибке попали в раздел "санкции": ')
print(df.loc[(df['y'] == 4) & (df['Категория'] != 'санкции')]['Заголовок'].to_list())
print()
print('Заголовки новостей, которые попали в смешнный второй кластер: ')
print(df.loc[(df['y'] == 2)]['Заголовок'].to_list())
print()

Заголовки новостей, которые по ошибке попали в раздел "поставки газа": 
['В Петербурге открыли «большой подарок от «Газпрома»', 'Газ запутался в платежах', 'Союзные газударства', '«"Газпром" практически ничего не теряет»', 'В ЕС сочли нарушением контракта отключение Польши и Болгарии от газа', '«Газпром» прекратит поставки Shell в Германии', 'Австрийская OMV оплатила российский газ по новой схеме']

Заголовки новостей, которые по ошибке попали в раздел "санкции": 
['Правительство временно разрешило поставки СПГ бывшей «дочке» «Газпрома»', 'Запасай, или потеряешь']

Заголовки новостей, которые попали в смешнный второй кластер: 
['«Эксперт 400»: для роста капитализации российских компаний необходимы сильные внутренние инвесторы', '«Эксперт РА» присвоил рейтинг компании «ГАЗПРОМ» на уровне ruAAA', 'Европа «перебрала» с запасами', '«Газпром» восьмой день не бронирует мощности газопровода Ямал—Европа', '«Газпром» снова не забронировал мощности газопровода Ямал—Европа', '«Газпром» перезаключ

Выводы:
- алгоритм работает близко к истинной разметке и правильно разделил большую часть новостей по кластерам 
- неправильно разделились новости, находившиеся на стыке категорий, особенно это заметно в путанице между категориями "санкции" и "поставки газа"
- второй кластер по сути можно объединить как "финансы". Если вчитаться в соответствующие новости, то можно заметить, что они относятся к бирже, инвестициям, оплате газа: всему, что можно отнести к финансовой части.

In [29]:
res = pd.DataFrame()
res['Article'] = df['Заголовок']
res['Cluster'] = df['y']
res.loc[(res['Cluster'] == 0), 'Cluster'] = 'рейтинг'
res.loc[(res['Cluster'] == 1), 'Cluster'] = 'поставки газа'
res.loc[(res['Cluster'] == 2), 'Cluster'] = 'финансы'
res.loc[(res['Cluster'] == 3), 'Cluster'] = 'события'
res.loc[(res['Cluster'] == 4), 'Cluster'] = 'санкции'

res['True category'] = df['Категория']
res

,Article,Cluster,True category
0,«Эксперт 400»: для роста капитализации российс...,финансы,инвестиции
1,«Эксперт РА» присвоил рейтинг компании «Газпро...,рейтинг,рейтинг
2,«Эксперт РА» присвоил рейтинг компании «ГАЗПРО...,финансы,рейтинг
3,Европа «перебрала» с запасами,финансы,добыча газа
4,«Газпром» восьмой день не бронирует мощности г...,финансы,поставки газа
...,...,...,...
84,«Газпром» подтвердил остановку поставок датско...,поставки газа,поставки газа
85,В Молдавии сообщили о снижении цены на российс...,финансы,поставки газа
86,«Газпром» вложился в бурение,финансы,биржа
87,«Газпром» поздравил Санкт-Петербург с 350-лети...,финансы,события


In [30]:
res.to_csv('events_clusters', encoding='utf-8', index=False)
